## Loading libraries

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

import os
import pandas as pd

import random

import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Read the dataset

In [ ]:
# Download the dataset
!wget https://www.dropbox.com/s/acabyl01j43cf9i/bbc.csv?dl=0 -O bbc.csv

--2021-02-10 17:56:52--  https://www.dropbox.com/s/acabyl01j43cf9i/bbc.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/acabyl01j43cf9i/bbc.csv [following]
--2021-02-10 17:56:52--  https://www.dropbox.com/s/raw/acabyl01j43cf9i/bbc.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4e6f91289e7d6299088e5c6a01.dl.dropboxusercontent.com/cd/0/inline/BIqPjVOfjjIpkh4qtPHV9Gf55kb8-_SoziRUwivfz8q_PsFU9y83DtAljiDTxrgMZ7deMg-9pTCDe7GbFPZnm5EFkCunhbRKHhgDcHq0Eo1hxfRYVetTFK-dxS2Ji8puOs8/file# [following]
--2021-02-10 17:56:52--  https://uc4e6f91289e7d6299088e5c6a01.dl.dropboxusercontent.com/cd/0/inline/BIqPjVOfjjIpkh4qtPHV9Gf55kb8-_SoziRUwivfz8q_PsFU9y83DtAljiDTxrgMZ7deMg-9pTCDe7GbFPZnm5EFkCunhbRKHhgDcHq0Eo1hxfRYVetTFK-dx

In [ ]:
df = pd.read_csv('bbc.csv', index_col=0)
df.head()

,Article,Class
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [ ]:
# since the dataset is not suffled, lets give it a shake
df = df.sample(frac = 1) 

# tokenize the sentences 
df['Article'] = [word_tokenize(line) for line in df['Article']]

# remove the stopwords
stop_words = set(stopwords.words('english'))
df['Article'] = [[w for w in line if not w in stop_words] for line in df['Article']]

# convert the categorical output to numerical format. (Since there are only 5 classes, we'll do it manually)
cat_to_num = {'politics': 0, 'business': 1, 'tech': 2, 'entertainment': 3, 'sport': 4}

df['Class'] = [cat_to_num[item] for item in df['Class']]
df.head()


,Article,Class
1701,"[Online, commons, spark, debate, Online, commu...",2
1470,"[Stam, spices, Man, Utd, encounter, AC, Milan,...",4
1620,"[Podcasts, mark, rise, DIY, radio, An, Apple, ...",2
1101,"[Blair, 's, hope, Blunkett, return, The, event...",0
1375,"[Holmes, urged, compete, Worlds, Jolanda, Cepl...",4


In [ ]:

# converting the dataframe to a list array
X_data, y_data = np.array(df['Article']), np.array( df['Class'] )


## Use glove embeddings

In [ ]:
# Download glove
# !wget http://nlp.stanford.edu/data/glove.6B.zip 
# !unzip glove.6B.zip

# using 100-dim Glove word embeddings

embeddings = {}
f = open('./glove.6B.100d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    embedding = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings[word] = embedding 
f.close()

print('GloVe data loaded')

GloVe data loaded


In [ ]:
# initializing unknown token (in case a word is not found, we'll assign it an unknown token)
UNK = np.zeros(100)
UNK[45] = 0.5

In [ ]:
X_data = np.array([np.array([embeddings[word] if word in embeddings.keys() else UNK for word in row]) for row in X_data])
len(X_data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


1912

In [ ]:
# max_seq_len = 100
an_padding = np.zeros(300)
max_seq_len = max([len(row) for row in X_data])
for idx, row in enumerate(X_data):
    X_data[idx] = np.pad(X_data[idx], ((0, max_seq_len - len(row)), (0, 0)), 'constant')
    X_data[idx] = np.vstack(X_data[idx]).astype(np.float)

In [ ]:
X_data[0].shape

(2981, 100)

In [ ]:
X_data_new = np.zeros((1912, 2981, 100), dtype=float)

for i, row in enumerate(X_data):
    for j, word in enumerate(row):
        for k, ele in enumerate(word):
            X_data_new[i][j][k] = ele

print(X_data_new.shape, X_data_new.dtype, y_data.dtype)

(1912, 2981, 100) float64 uint8


In [ ]:
X_data_new = X_data_new[:, :300]
X_data_new.shape

(1912, 300, 100)

In [ ]:
# setting fixed seed value to get reproducible results (however, note that the dataset shuffle is not seeded, and hence the results would indeed vary)

seed_value = 1
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
point1, point2 = len(y_data) // 3, 2 * len(y_data) // 3
point1, point2

(637, 1274)

In [ ]:
max_seq_len = 300 # since now we are using saved dataset (embeddings)

In [ ]:
# Note that the 3-fold and 70-10-20 split in the assignment are conflicting; 
# we have taken the liberty to make the split as 66-11-22 instead

## 1st of 3 folds

**Process train data**

In [ ]:
val_cut_point = point2 + (len(y_data) - point2) // 3

X_train, X_val, X_test = X_data_new[:point2], X_data_new[point2:val_cut_point], X_data_new[val_cut_point:]
y_train, y_val, y_test = y_data[:point2], y_data[point2:val_cut_point], y_data[val_cut_point:]

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
batch_size = 32
train_data = train_data.batch(batch_size)
val_data = val_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [ ]:
train_data = train_data.prefetch(1)

for x_batch, y_batch in train_data.take(2):
    print(x_batch.shape, y_batch.shape)

(32, 300, 100) (32,)
(32, 300, 100) (32,)


***train GRU***

In [ ]:
# This is the default GRU, hence it works using 'tanh' as the underlying activation function

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)
print( model.summary() )

Epoch 1/5
40/40 [==============================] - 62s 1s/step - loss: 1.5332 - accuracy: 0.3310 - val_loss: 1.4264 - val_accuracy: 0.3726
Epoch 2/5
40/40 [==============================] - 56s 1s/step - loss: 1.3352 - accuracy: 0.4413 - val_loss: 1.2910 - val_accuracy: 0.4434
Epoch 3/5
40/40 [==============================] - 56s 1s/step - loss: 1.2231 - accuracy: 0.4761 - val_loss: 1.1276 - val_accuracy: 0.5896
Epoch 4/5
40/40 [==============================] - 56s 1s/step - loss: 1.1979 - accuracy: 0.5763 - val_loss: 1.1608 - val_accuracy: 0.5094
Epoch 5/5
40/40 [==============================] - 56s 1s/step - loss: 1.2282 - accuracy: 0.5259 - val_loss: 1.2274 - val_accuracy: 0.5566
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 300, 128)          88320     
_________________________________________________________________
gru_5 (GRU)        

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.5105882352941177
              precision    recall  f1-score   support

           0       0.49      0.69      0.57        89
           1       0.70      0.47      0.56       119
           2       0.65      0.34      0.44        95
           3       0.40      0.87      0.54        78
           4       0.00      0.00      0.00        44

    accuracy                           0.51       425
   macro avg       0.45      0.47      0.42       425
weighted avg       0.52      0.51      0.48       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Train LSTM**

In [ ]:
# This is the default LSTM, hence it works using 'tanh' as the underlying activation function

model = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)
model.summary()

Epoch 1/5
40/40 [==============================] - 64s 1s/step - loss: 1.5068 - accuracy: 0.3539 - val_loss: 1.3170 - val_accuracy: 0.4434
Epoch 2/5
40/40 [==============================] - 59s 1s/step - loss: 1.1647 - accuracy: 0.5131 - val_loss: 0.8978 - val_accuracy: 0.6038
Epoch 3/5
40/40 [==============================] - 59s 1s/step - loss: 1.1264 - accuracy: 0.5863 - val_loss: 1.0691 - val_accuracy: 0.5991
Epoch 4/5
40/40 [==============================] - 59s 1s/step - loss: 1.3329 - accuracy: 0.4720 - val_loss: 1.3950 - val_accuracy: 0.4387
Epoch 5/5
40/40 [==============================] - 60s 1s/step - loss: 1.3134 - accuracy: 0.4378 - val_loss: 1.0869 - val_accuracy: 0.5189
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 300, 128)          117248    
_________________________________________________________________
gru_6 (GRU)        

In [ ]:
Y_pred= model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.5539906103286385
              precision    recall  f1-score   support

           0       0.74      0.46      0.57        84
           1       0.39      0.90      0.54       117
           2       0.92      0.84      0.88        96
           3       0.64      0.08      0.15        85
           4       1.00      0.09      0.17        44

    accuracy                           0.55       426
   macro avg       0.74      0.48      0.46       426
weighted avg       0.69      0.55      0.51       426



## 2nd of 3 folds

**Process train data**

In [ ]:
val_cut_point = point1 // 3

X_train, X_val, X_test = X_data_new[point1:], X_data_new[:val_cut_point], X_data_new[val_cut_point:point1]
y_train, y_val, y_test = y_data[point1:], y_data[:val_cut_point], y_data[val_cut_point:point1]

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
batch_size = 32
train_data = train_data.batch(batch_size)
val_data = val_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [ ]:
train_data = train_data.prefetch(1)

for x_batch, y_batch in train_data.take(2):
    print(x_batch.shape, y_batch.shape)

(32, 300, 100) (32,)
(32, 300, 100) (32,)


**train GRU**

In [ ]:
# This is the default GRU, hence it works using 'tanh' as the underlying activation function

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)
print(model.summary())

Epoch 1/5
40/40 [==============================] - 61s 1s/step - loss: 1.5098 - accuracy: 0.3222 - val_loss: 1.3535 - val_accuracy: 0.4245
Epoch 2/5
40/40 [==============================] - 56s 1s/step - loss: 1.2714 - accuracy: 0.4600 - val_loss: 1.1750 - val_accuracy: 0.4953
Epoch 3/5
40/40 [==============================] - 56s 1s/step - loss: 1.1822 - accuracy: 0.5144 - val_loss: 1.2269 - val_accuracy: 0.4717
Epoch 4/5
40/40 [==============================] - 56s 1s/step - loss: 1.3327 - accuracy: 0.4735 - val_loss: 1.3252 - val_accuracy: 0.4434
Epoch 5/5
40/40 [==============================] - 56s 1s/step - loss: 1.1729 - accuracy: 0.5053 - val_loss: 0.9487 - val_accuracy: 0.5613
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 300, 128)          88320     
_________________________________________________________________
gru_8 (GRU)        

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.508235294117647
              precision    recall  f1-score   support

           0       0.72      0.60      0.65       102
           1       0.55      0.21      0.30       114
           2       0.92      0.84      0.88        82
           3       0.28      0.77      0.41        81
           4       0.00      0.00      0.00        46

    accuracy                           0.51       425
   macro avg       0.49      0.48      0.45       425
weighted avg       0.55      0.51      0.49       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**train LSTM**

In [ ]:
# This is the default LSTM, hence it works using 'tanh' as the underlying activation function

model = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
print(model.summary())
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 300, 128)          117248    
_________________________________________________________________
gru_9 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
Total params: 216,965
Trainable params: 216,965
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 65s 2s/step - loss: 1.4842 - accuracy: 0.3488 - val_loss: 1.3244 - val_accuracy: 0.5519
Epoch 2/5
40/40 [==============================] - 60s 1s/step - loss: 1.3126 - accuracy: 0.4695 - val_loss: 1.0219 - val_accuracy: 0.5613
Epoch 3/5
40/40 [==============================] - 60s 1s/step - loss: 1.1748 - accuracy: 0.494

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.788235294117647
              precision    recall  f1-score   support

           0       0.74      0.76      0.75       102
           1       0.83      0.97      0.90       114
           2       0.92      0.93      0.92        82
           3       0.68      0.78      0.73        81
           4       0.58      0.15      0.24        46

    accuracy                           0.79       425
   macro avg       0.75      0.72      0.71       425
weighted avg       0.77      0.79      0.76       425



## 3rd of 3 folds

**process train dataset**

In [ ]:
val_cut_point = point1 + (point2 - point1) // 3

np.array(np.concatenate((y_data[:point1], y_data[point2:]), axis=0))

X_train, X_val, X_test = np.array(np.concatenate((X_data_new[:point1], X_data_new[point2:]), axis=0)), X_data_new[point1:val_cut_point], X_data_new[val_cut_point:point2]
y_train, y_val, y_test = np.array(np.concatenate((y_data[:point1], y_data[point2:]), axis=0)), y_data[point1:val_cut_point], y_data[val_cut_point:point2]

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
batch_size = 32
train_data = train_data.batch(batch_size)
val_data = val_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [ ]:
train_data = train_data.prefetch(1)

for x_batch, y_batch in train_data.take(2):
    print(x_batch.shape, y_batch.shape)

(32, 300, 100) (32,)
(32, 300, 100) (32,)


**train GRU**

In [ ]:
# This is the default GRU, hence it works using 'tanh' as the underlying activation function

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
print(model.summary())
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, 300, 128)          88320     
_________________________________________________________________
gru_11 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 645       
Total params: 188,037
Trainable params: 188,037
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 61s 1s/step - loss: 1.5292 - accuracy: 0.3298 - val_loss: 1.2962 - val_accuracy: 0.4198
Epoch 2/5
40/40 [==============================] - 57s 1s/step - loss: 1.3387 - accuracy: 0.4426 - val_loss: 1.1174 - val_accuracy: 0.4906
Epoch 3/5
40/40 [==============================] - 57s 1s/step - loss: 1.1826 - accuracy: 0.511

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.7058823529411765
              precision    recall  f1-score   support

           0       0.77      0.49      0.60        89
           1       0.64      0.97      0.77       119
           2       0.94      0.86      0.90        95
           3       0.57      0.71      0.63        78
           4       1.00      0.07      0.13        44

    accuracy                           0.71       425
   macro avg       0.78      0.62      0.61       425
weighted avg       0.76      0.71      0.67       425



**train LSTM**

In [ ]:
# This is the default LSTM, hence it works using 'tanh' as the underlying activation function

model = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
print(model.summary())
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 300, 128)          117248    
_________________________________________________________________
gru_12 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 645       
Total params: 216,965
Trainable params: 216,965
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 65s 2s/step - loss: 1.5155 - accuracy: 0.3519 - val_loss: 1.4323 - val_accuracy: 0.4009
Epoch 2/5
40/40 [==============================] - 60s 1s/step - loss: 1.2103 - accuracy: 0.4875 - val_loss: 1.2831 - val_accuracy: 0.4717
Epoch 3/5
40/40 [==============================] - 59s 1s/step - loss: 1.1036 - accuracy: 0.552

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.5341176470588235
              precision    recall  f1-score   support

           0       0.58      0.29      0.39        89
           1       0.52      0.94      0.67       119
           2       0.49      0.33      0.39        95
           3       0.57      0.71      0.63        78
           4       0.50      0.07      0.12        44

    accuracy                           0.53       425
   macro avg       0.53      0.47      0.44       425
weighted avg       0.53      0.53      0.49       425



## Running model with various activation functions (using data split of 1st fold)

**GRU with RELU**

In [ ]:
# GRU with ReLU

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2, activation='relu'), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2, activation='relu'),
    keras.layers.Dense(5, activation="softmax")
])

print(model.summary())
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_13 (GRU)                 (None, 300, 128)          88320     
_________________________________________________________________
gru_14 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 645       
Total params: 188,037
Trainable params: 188,037
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 62s 1s/step - loss: 1.5614 - accuracy: 0.3182 - val_loss: 1.4415 - val_accuracy: 0.3113
Epoch 2/5
40/40 [==============================] - 57s 1s/step - loss: 1.4362 - accuracy: 0.3835 - val_loss: 1.3293 - val_accuracy: 0.4528
Epoch 3/5
40/40 [==============================] - 56s 1s/step - loss: 1.3688 - accuracy: 0.457

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.5764705882352941
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        89
           1       0.40      0.75      0.52       119
           2       0.88      0.98      0.93        95
           3       0.56      0.13      0.21        78
           4       0.00      0.00      0.00        44

    accuracy                           0.58       425
   macro avg       0.50      0.49      0.46       425
weighted avg       0.55      0.58      0.52       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**LSTM with RELU**

In [ ]:
# LSTM with ReLU

model = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2, activation='relu'), # none - batch_size
    keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, activation='relu'),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
print(model.summary())
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 300, 128)          117248    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 645       
Total params: 249,477
Trainable params: 249,477
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 68s 2s/step - loss: 7.6364 - accuracy: 0.3142 - val_loss: 1.5470 - val_accuracy: 0.3160
Epoch 2/5
40/40 [==============================] - 63s 2s/step - loss: 1.5427 - accuracy: 0.3288 - val_loss: 1.5282 - val_accuracy: 0.3160
Epoch 3/5
40/40 [==============================] - 63s 2s/step - loss: 1.5282 - accuracy: 0.329

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.3247058823529412
              precision    recall  f1-score   support

           0       0.32      0.64      0.42        89
           1       0.33      0.68      0.45       119
           2       0.00      0.00      0.00        95
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00        44

    accuracy                           0.32       425
   macro avg       0.13      0.26      0.17       425
weighted avg       0.16      0.32      0.21       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**GRU With Sigmoid**

In [ ]:
# GRU with Sigmoid

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2, activation='sigmoid'), # none - batch_size
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2, activation='sigmoid'),
    keras.layers.Dense(5, activation="softmax")
])

print(model.summary())
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_15 (GRU)                 (None, 300, 128)          88320     
_________________________________________________________________
gru_16 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 645       
Total params: 188,037
Trainable params: 188,037
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 62s 1s/step - loss: 1.6102 - accuracy: 0.2475 - val_loss: 1.5593 - val_accuracy: 0.2642
Epoch 2/5
40/40 [==============================] - 57s 1s/step - loss: 1.5733 - accuracy: 0.2694 - val_loss: 1.5074 - val_accuracy: 0.3160
Epoch 3/5
40/40 [==============================] - 57s 1s/step - loss: 1.5313 - accuracy: 0.31

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.5247058823529411
              precision    recall  f1-score   support

           0       0.83      0.39      0.53        89
           1       0.38      0.97      0.54       119
           2       0.97      0.76      0.85        95
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00        44

    accuracy                           0.52       425
   macro avg       0.44      0.43      0.39       425
weighted avg       0.50      0.52      0.45       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**LSTM with Sigmoid**

In [ ]:
# LSTM with sigmoid

model = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2, activation='sigmoid'),
    keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, activation='sigmoid'),
    keras.layers.Dense(5, activation="softmax")
])
print(model.summary())
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 300, 128)          117248    
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 645       
Total params: 249,477
Trainable params: 249,477
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 69s 2s/step - loss: 1.6240 - accuracy: 0.2293 - val_loss: 1.5593 - val_accuracy: 0.2642
Epoch 2/5
40/40 [==============================] - 64s 2s/step - loss: 1.5797 - accuracy: 0.2756 - val_loss: 1.5139 - val_accuracy: 0.3019
Epoch 3/5
40/40 [==============================] - 63s 2s/step - loss: 1.5414 - accuracy: 0.31

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.3952941176470588
              precision    recall  f1-score   support

           0       0.40      0.02      0.04        89
           1       0.36      0.92      0.52       119
           2       0.49      0.60      0.54        95
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00        44

    accuracy                           0.40       425
   macro avg       0.25      0.31      0.22       425
weighted avg       0.29      0.40      0.27       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**GRU with ELU**

In [ ]:
# GRU with ELU

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2, activation='elu'),
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2, activation='elu'),
    keras.layers.Dense(5, activation="softmax")
])


model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
print(model.summary())
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_17 (GRU)                 (None, 300, 128)          88320     
_________________________________________________________________
gru_18 (GRU)                 (None, 128)               99072     
_________________________________________________________________
dense_12 (Dense)             (None, 5)                 645       
Total params: 188,037
Trainable params: 188,037
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 61s 1s/step - loss: 1.5481 - accuracy: 0.3262 - val_loss: 1.3017 - val_accuracy: 0.4245
Epoch 2/5
40/40 [==============================] - 57s 1s/step - loss: 1.3406 - accuracy: 0.4291 - val_loss: 1.1186 - val_accuracy: 0.5142
Epoch 3/5
40/40 [==============================] - 57s 1s/step - loss: 1.2019 - accuracy: 0.50

In [ ]:
Y_pred = model.predict_classes(X_test)
print(accuracy_score(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.7858823529411765
              precision    recall  f1-score   support

           0       0.94      0.75      0.84        89
           1       0.67      0.89      0.77       119
           2       0.94      0.93      0.93        95
           3       0.70      0.81      0.75        78
           4       0.83      0.23      0.36        44

    accuracy                           0.79       425
   macro avg       0.82      0.72      0.73       425
weighted avg       0.81      0.79      0.77       425



**LSTM with ELU**

In [ ]:
# LSTM with elu

model = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2, activation='elu'),
    keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, activation='elu'),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
print(model.summary())
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 300, 128)          117248    
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 645       
Total params: 249,477
Trainable params: 249,477
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 69s 2s/step - loss: nan - accuracy: 0.3433 - val_loss: nan - val_accuracy: 0.2594
Epoch 2/5
40/40 [==============================] - 64s 2s/step - loss: nan - accuracy: 0.2204 - val_loss: nan - val_accuracy: 0.2594
Epoch 3/5
40/40 [==============================] - 63s 2s/step - loss: nan - accuracy: 0.2204 - val_loss: 

In [ ]:
Y_pred_lstm_elu = model.predict_classes(X_test)
print(classification_report(y_test, Y_pred_lstm_elu))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.21      1.00      0.35        89
           1       0.00      0.00      0.00       119
           2       0.00      0.00      0.00        95
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00        44

    accuracy                           0.21       425
   macro avg       0.04      0.20      0.07       425
weighted avg       0.04      0.21      0.07       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**GRU with LeakyRELU**

In [ ]:
# GRU with LeakyRELU

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.LeakyReLU(),
    keras.layers.GRU(128, dropout=0.2, recurrent_dropout=0.2, activation='elu'),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
print(model.summary())
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_19 (GRU)                 (None, 300, 128)          88320     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300, 128)          0         
_________________________________________________________________
gru_20 (GRU)                 (None, 128)               99072     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 645       
Total params: 188,037
Trainable params: 188,037
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
40/40 [==============================] - 62s 1s/step - loss: 1.5521 - accuracy: 0.3345 - val_loss:

In [ ]:
Y_pred = model.predict_classes(X_test)
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.95      0.61      0.74        89
           1       0.80      0.89      0.84       119
           2       0.88      0.98      0.93        95
           3       0.48      0.79      0.60        78
           4       0.00      0.00      0.00        44

    accuracy                           0.74       425
   macro avg       0.62      0.65      0.62       425
weighted avg       0.71      0.74      0.71       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**LSTM with LeakyRELU**

In [ ]:
# LSTM with LeakyRELU

model = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=[max_seq_len, 100], dropout=0.2, recurrent_dropout=0.2),
    keras.layers.LeakyReLU(),
    keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2, activation='elu'),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(5, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics="accuracy")
history = model.fit(train_data, epochs=5, validation_data=val_data, batch_size=32)

Epoch 1/5
40/40 [==============================] - 68s 2s/step - loss: 1.6293 - accuracy: 0.3512 - val_loss: 1.5430 - val_accuracy: 0.3255
Epoch 2/5
40/40 [==============================] - 64s 2s/step - loss: 1.5201 - accuracy: 0.3270 - val_loss: 1.4978 - val_accuracy: 0.3255
Epoch 3/5
40/40 [==============================] - 64s 2s/step - loss: 1.4882 - accuracy: 0.3368 - val_loss: 1.4732 - val_accuracy: 0.3302
Epoch 4/5
40/40 [==============================] - 64s 2s/step - loss: 1.4738 - accuracy: 0.3470 - val_loss: 1.4605 - val_accuracy: 0.3396
Epoch 5/5
40/40 [==============================] - 64s 2s/step - loss: 1.4698 - accuracy: 0.3541 - val_loss: 1.4515 - val_accuracy: 0.3491


In [ ]:
Y_pred = model.predict_classes(X_test)
print(classification_report(y_test, Y_pred))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.35      0.65      0.45        89
           1       0.34      0.68      0.45       119
           2       0.80      0.17      0.28        95
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00        44

    accuracy                           0.36       425
   macro avg       0.30      0.30      0.24       425
weighted avg       0.35      0.36      0.28       425



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
